# celverapi

> Library that allows the user to forecast multipletime Timeseries, parallel, using multiprocessing 

## Install

`pip install celverapi`

## How to use the Forecasting API 

In this example we are going to use a DataFrame from 4 different grocery stores. 
The idea is to Forecast various models, from every single store, simultaneously 

In [ ]:
from celverapi.core import *
from celverapi.main import *
import pandas as pd

In [ ]:
df = pd.read_parquet('Data/example.parquet')
#df.head(10)

Using the function unique_dataframe_values(), we are going to create a list of DataFrames from every unique ID value in the Main DataFrame. 
In this case, we need a DataFrame from every Store.

In [ ]:
list_of_DataFrames = unique_dataframe_values('store_nbr')
list_of_DataFrames

[            date  store_nbr     family   sales  onpromotion
 0     2013-01-01          6  GROCERY I     0.0          0.0
 4     2013-01-02          6  GROCERY I  5535.0          0.0
 8     2013-01-03          6  GROCERY I  4040.0          0.0
 12    2013-01-04          6  GROCERY I  3314.0          0.0
 16    2013-01-05          6  GROCERY I  4857.0          0.0
 ...          ...        ...        ...     ...          ...
 6716  2017-08-11          6  GROCERY I  4466.0        776.0
 6720  2017-08-12          6  GROCERY I  4027.0        770.0
 6724  2017-08-13          6  GROCERY I  5481.0        817.0
 6728  2017-08-14          6  GROCERY I  4142.0        773.0
 6732  2017-08-15          6  GROCERY I  4334.0        779.0
 
 [1684 rows x 5 columns],
             date  store_nbr     family   sales  onpromotion
 1     2013-01-01          7  GROCERY I     0.0          0.0
 5     2013-01-02          7  GROCERY I  4172.0          0.0
 9     2013-01-03          7  GROCERY I  3279.0          

Once we have a list with the DataFrames that we want to work with, we are going to proceed splitting them into a Train and Test DataFrame.
In this case the Train DF is going to be from the Start Date until the 31-07-2017.

In [ ]:
list_of_trained_df, list_of_tested = split_into_train_test_dataframe(list_of_DataFrames, "date", "2017-08-01")
list_of_trained_df

[            date  store_nbr     family   sales  onpromotion
 0     2013-01-01          6  GROCERY I     0.0          0.0
 4     2013-01-02          6  GROCERY I  5535.0          0.0
 8     2013-01-03          6  GROCERY I  4040.0          0.0
 12    2013-01-04          6  GROCERY I  3314.0          0.0
 16    2013-01-05          6  GROCERY I  4857.0          0.0
 ...          ...        ...        ...     ...          ...
 6656  2017-07-27          6  GROCERY I  3820.0        765.0
 6660  2017-07-28          6  GROCERY I  4602.0        807.0
 6664  2017-07-29          6  GROCERY I  6050.0        817.0
 6668  2017-07-30          6  GROCERY I  6664.0        840.0
 6672  2017-07-31          6  GROCERY I  5237.0        806.0
 
 [1669 rows x 5 columns],
             date  store_nbr     family   sales  onpromotion
 1     2013-01-01          7  GROCERY I     0.0          0.0
 5     2013-01-02          7  GROCERY I  4172.0          0.0
 9     2013-01-03          7  GROCERY I  3279.0          

After that, we will convert this DataFrames into TimeSeries objects and save them in their respective lists. 

In [ ]:
trained_timeseries, valed_timeseries = converting_dataframes_into_objects(list_of_trained_df, list_of_tested, 'date', 'store_nbr', 'sales')
trained_timeseries

[          date  store_nbr     family   sales  onpromotion
 0   2013-01-01          6  GROCERY I     0.0          0.0
 4   2013-01-02          6  GROCERY I  5535.0          0.0
 8   2013-01-03          6  GROCERY I  4040.0          0.0
 12  2013-01-04          6  GROCERY I  3314.0          0.0
 16  2013-01-05          6  GROCERY I  4857.0          0.0,
           date  store_nbr     family   sales  onpromotion
 1   2013-01-01          7  GROCERY I     0.0          0.0
 5   2013-01-02          7  GROCERY I  4172.0          0.0
 9   2013-01-03          7  GROCERY I  3279.0          0.0
 13  2013-01-04          7  GROCERY I  2681.0          0.0
 17  2013-01-05          7  GROCERY I  2662.0          0.0,
           date  store_nbr     family   sales  onpromotion
 2   2013-01-01          8  GROCERY I     0.0          0.0
 6   2013-01-02          8  GROCERY I  5277.0          0.0
 10  2013-01-03          8  GROCERY I  3783.0          0.0
 14  2013-01-04          8  GROCERY I  3481.0         

The last part of the preparation, is to create a dictionary that maps the ID from the TimeSeries object with the ForecastingTask Object. 


In [ ]:
dict_of_forecasting_tasks = creating_forecasting_task_dict(trained_timeseries, valed_timeseries, forecast_horizon=15)
dict_of_forecasting_tasks

{6: <celverapi.core.ForecastingTask at 0x1b36fdf2b20>,
 7: <celverapi.core.ForecastingTask at 0x1b36fdf2af0>,
 8: <celverapi.core.ForecastingTask at 0x1b36fdf2a30>,
 9: <celverapi.core.ForecastingTask at 0x1b36fdf2f70>}

To Forecast this TimeSeries, we need to call the forecast() function, where we need to pass a list of the models that we want to use and the Forecasting Task dictionary.


In [ ]:
if __name__ == "__main__":
    final_result = forecast(['ets', 'naive', 'trend'], dict_of_forecasting_tasks)

#final_result 
#Removing DF for index HTML

## How to use the Weather API 

In [ ]:
from celverapi.weahter import * 

First of all, you will need to assign your API Key from Visual Crosssing to the ApiKey variable 

In [ ]:
ApiKey='37QNHAHY6SEXQ4N3R8MMQYZ3T'

In order to get a DataFrame from a particular Weather Forecast, you just need to call the get_forecast() function with the required parameters and assing it to a variable.
In this particular example, we are going to get the Weather Forecast for the City of Hamburg from 17-05-2022 until 22-05-2022

In [ ]:
dataframe = get_forecast("Hamburg", "2022-05-17", "2022-05-22")
dataframe

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
0,2022-05-17,1652738400,18.3,13.3,15.3,18.3,13.3,15.3,13.3,88.1,...,05:16:23,1652757383,21:17:36,1652815056,0.52,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"[EDHI, EDDH, EDHL, C7997]",comb
1,2022-05-18,1652824800,25.3,11.0,17.8,25.3,11.0,17.8,13.8,79.0,...,05:14:51,1652843691,21:19:12,1652901552,0.54,"Rain, Partially cloudy",Partly cloudy throughout the day with storms p...,rain,None,fcst
2,2022-05-19,1652911200,25.2,13.1,18.5,25.2,13.1,18.5,15.9,86.2,...,05:13:20,1652930000,21:20:47,1652988047,0.58,"Rain, Overcast",Cloudy skies throughout the day with storms po...,rain,None,fcst
3,2022-05-20,1652997600,18.6,10.0,14.0,18.6,9.4,13.9,12.1,88.8,...,05:11:52,1653016312,21:22:21,1653074541,0.63,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,None,fcst
4,2022-05-21,1653084000,16.8,9.1,12.7,16.8,6.8,12.2,10.1,85.1,...,05:10:27,1653102627,21:23:53,1653161033,0.68,"Rain, Partially cloudy",Partly cloudy throughout the day with rain in ...,rain,None,fcst
5,2022-05-22,1653170400,19.8,8.1,13.8,19.8,7.2,13.5,9.0,75.5,...,05:09:04,1653188944,21:25:24,1653247524,0.74,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,None,fcst
